In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.metrics import mean_absolute_error
from ISLP import load_data, confusion_table

In [139]:
or_df = pd.read_csv("OR_quarterly_aggregation.csv")
wa_df = pd.read_csv("WA_Hospital_Financial_Quarterly_Aggregate_Report_20250520.csv")

In [143]:
or_df['State'] = 'Oregon'
wa_df['State'] = 'Washington'

In [149]:
or_key_cols = ['Hospital Name', 'Quarter', 'Year', 'State']
wa_key_cols = ['Hospital_Name', 'Quarter', 'Year', 'State']

or_days_cols = [col for col in or_df.columns if 'Days' in col]
wa_days_cols = [col for col in wa_df.columns if 'Days' in col]

or_cols = or_key_cols + or_days_cols
wa_cols = wa_key_cols + wa_days_cols

oregon = or_df[or_cols]
washington = wa_df[wa_cols]

oregon.head()

,Hospital Name,Quarter,Year,State,Patient Days of Acute Inpatient (Medicare),Patient Days of Acute Inpatient (Medicaid),Patient Days of Acute Inpatient (Self Pay),Patient Days of Acute Inpatient (Commercial),Patient Days of Acute Inpatient (Others),Patient Days of Acute Inpatient,...,Patient Days of DPU (Commercial),Patient Days of DPU (Others),Patient Days of DPU,Total Patient Days (Medicare),Total Patient Days (Medicaid),Total Patient Days (Self Pay),Total Patient Days (Commercial),Total Patient Days (Others),Total Patient Days,Newborn Patient Days
0,Legacy Emanuel Medical Center,1,2007,Oregon,5732.0,5863.0,1831.0,0.0,9940.0,23366.0,...,0.0,1245.0,2702.0,6137.0,6814.0,1932.0,0.0,11185.0,26068.0,833.0
1,Legacy Emanuel Medical Center,2,2007,Oregon,5365.0,7164.0,1907.0,0.0,10239.0,24675.0,...,0.0,1461.0,2894.0,5707.0,8194.0,1968.0,0.0,11700.0,27569.0,777.0
2,Legacy Emanuel Medical Center,3,2007,Oregon,5139.0,6423.0,2116.0,0.0,10917.0,24595.0,...,0.0,1192.0,2706.0,5598.0,7366.0,2228.0,0.0,12109.0,27301.0,872.0
3,Legacy Emanuel Medical Center,4,2007,Oregon,5456.0,5761.0,1520.0,0.0,11018.0,23755.0,...,0.0,1351.0,2669.0,5818.0,6595.0,1642.0,0.0,12369.0,26424.0,805.0
4,Legacy Emanuel Medical Center,1,2008,Oregon,5724.0,6922.0,1364.0,0.0,10209.0,24219.0,...,0.0,1453.0,2695.0,5996.0,7761.0,1495.0,0.0,11662.0,26914.0,860.0


In [177]:
column_mappings = {
    'Hospital Name': 'Hospital_Name',
   'Patient Days of Acute Inpatient (Medicare)': 'Acute_Care_Medicare_Patient_Days',
   'Patient Days of Acute Inpatient (Medicaid)': 'Acute_Care_Medicaid_Patient_Days',
   'Patient Days of Acute Inpatient (Self Pay)': 'Acute_Care_Self_Pay_Patient_Days',
   'Patient Days of Acute Inpatient (Commercial)': 'Acute_Care_Commercial_Patient_Days',
   'Patient Days of Acute Inpatient (Others)': 'Acute_Care_Other_Patient_Days',
   'Patient Days of Acute Inpatient': 'Acute_Care_Total_Patient_Days',
   'Patient Days of Swing Bed (Medicare)': 'Swing_Bed_Medicare_Patient_Days',
   'Patient Days of Swing Bed (Medicaid)': 'Swing_Bed_Medicaid_Patient_Days',
   'Patient Days of Swing Bed (Self Pay)': 'Swing_Bed_Self_Pay_Patient_Days',
   'Patient Days of Swing Bed (Commercial)': 'Swing_Bed_Commercial_Patient_Days',
   'Patient Days of Swing Bed (Others)': 'Swing_Bed_Other_Patient_Days',
   'Patient Days of Swing Bed': 'Swing_Bed_Total_Patient_Days',
   'Patient Days of Subacute & LTC (Medicare)': 'SNF_Medicare_Patient_Days',
   'Patient Days of Subacute & LTC (Medicaid)': 'SNF_Medicaid_Patient_Days',
   'Patient Days of Subacute & LTC (Self Pay)': 'SNF_Self_Pay_Patient_Days',
   'Patient Days of Subacute & LTC (Commercial)': 'SNF_Commercial_Patient_Days',
   'Patient Days of Subacute & LTC (Others)': 'SNF_Other_Patient_Days',
   'Patient Days of Subacute & LTC': 'SNF_Total_Patient_Days',
   'Patient Days of DPU (Medicare)': 'Psych_Rehab_CDU_Medicare_Patient_Days',
   'Patient Days of DPU (Medicaid)': 'Psych_Rehab_CDU_Medicaid_Patient_Days',
   'Patient Days of DPU (Self Pay)': 'Psych_Rehab_CDU_Self_Pay_Patient_Days',
   'Patient Days of DPU (Commercial)': 'Psych_Rehab_CDU_Commercial_Patient_Days',
   'Patient Days of DPU (Others)': 'Psych_Rehab_CDU_Other_Patient_Days',
   'Patient Days of DPU': 'Psych_Rehab_CDU_Total_Patient_Days',
   'Total Patient Days (Medicare)': 'Total_Medicare_Patient_Days',
   'Total Patient Days (Medicaid)': 'Total_Medicaid_Patient_Days',
   'Total Patient Days (Self Pay)': 'Total_Self_Pay_Patient_Days',
   'Total Patient Days (Commercial)': 'Total_Commercial_Patient_Days',
   'Total Patient Days (Others)': 'Total_Other_Patient_Days',
   'Total Patient Days': 'Total_Patient_Days',
   'Newborn Patient Days': 'Total_Newborn_Days'
}

oregon_renamed = oregon.rename(columns = column_mappings)
oregon_renamed.columns

Index(['Hospital_Name', 'Quarter', 'Year', 'State',
       'Acute_Care_Medicare_Patient_Days', 'Acute_Care_Medicaid_Patient_Days',
       'Acute_Care_Self_Pay_Patient_Days',
       'Acute_Care_Commercial_Patient_Days', 'Acute_Care_Other_Patient_Days',
       'Acute_Care_Total_Patient_Days', 'Swing_Bed_Medicare_Patient_Days',
       'Swing_Bed_Medicaid_Patient_Days', 'Swing_Bed_Self_Pay_Patient_Days',
       'Swing_Bed_Commercial_Patient_Days', 'Swing_Bed_Other_Patient_Days',
       'Swing_Bed_Total_Patient_Days', 'SNF_Medicare_Patient_Days',
       'SNF_Medicaid_Patient_Days', 'SNF_Self_Pay_Patient_Days',
       'SNF_Commercial_Patient_Days', 'SNF_Other_Patient_Days',
       'SNF_Total_Patient_Days', 'Psych_Rehab_CDU_Medicare_Patient_Days',
       'Psych_Rehab_CDU_Medicaid_Patient_Days',
       'Psych_Rehab_CDU_Self_Pay_Patient_Days',
       'Psych_Rehab_CDU_Commercial_Patient_Days',
       'Psych_Rehab_CDU_Other_Patient_Days',
       'Psych_Rehab_CDU_Total_Patient_Days', 'Total_Medica

In [203]:
combined_df = pd.concat([washington, oregon_renamed], ignore_index = True, sort = False)
combined_df = combined_df.sort_values(['Year', 'Hospital_Name'], ascending = [True, True])
combined_df.head()

,Hospital_Name,Quarter,Year,State,Acute_Care_Medicare_Patient_Days,Acute_Care_Medicaid_Patient_Days,Acute_Care_Self_Pay_Patient_Days,Acute_Care_Commercial_Patient_Days,Acute_Care_Other_Patient_Days,Acute_Care_Total_Patient_Days,...,Psych_Rehab_CDU_Commercial_Patient_Days,Psych_Rehab_CDU_Other_Patient_Days,Psych_Rehab_CDU_Total_Patient_Days,Total_Medicare_Patient_Days,Total_Medicaid_Patient_Days,Total_Self_Pay_Patient_Days,Total_Commercial_Patient_Days,Total_Other_Patient_Days,Total_Patient_Days,Total_Newborn_Days
6884,Adventist Health Columbia Gorge Medical Center,1,2007,Oregon,952.0,367.0,91.0,0.0,498.0,1908.0,...,0.0,0.0,0.0,952.0,367.0,91.0,0.0,498.0,1908.0,199.0
6885,Adventist Health Columbia Gorge Medical Center,2,2007,Oregon,869.0,284.0,113.0,0.0,494.0,1760.0,...,0.0,0.0,0.0,869.0,284.0,113.0,0.0,494.0,1760.0,178.0
6886,Adventist Health Columbia Gorge Medical Center,3,2007,Oregon,921.0,339.0,143.0,0.0,582.0,1985.0,...,0.0,0.0,0.0,921.0,339.0,143.0,0.0,582.0,1985.0,200.0
6887,Adventist Health Columbia Gorge Medical Center,4,2007,Oregon,927.0,299.0,120.0,0.0,501.0,1847.0,...,0.0,0.0,0.0,927.0,299.0,120.0,0.0,501.0,1847.0,171.0
5900,Adventist Health Portland Medical Center,1,2007,Oregon,4036.0,1177.0,304.0,0.0,5789.0,11306.0,...,0.0,0.0,1909.0,5181.0,1917.0,328.0,0.0,5789.0,13215.0,653.0


In [205]:
combined_df.to_csv('Patient_Days.csv', index = False)